<a href="https://colab.research.google.com/github/Nithyarajoman/Machine-learning-Trial/blob/main/Resnet%2018%20Calculation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install image-classifiers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 50 kB 3.0 MB/s 


In [ ]:
# defined function (pip install image-classifiers)
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.applications import imagenet_utils
from keras.models import Sequential, Model

from classification_models.keras import Classifiers


In [ ]:
ResNet18, preprocess_input = Classifiers.get('resnet18')

In [ ]:
input_x, input_y = (224, 224)
model = ResNet18(include_top=False,
                        input_tensor=None,
                        input_shape=(
                            input_x, input_y, 3),
                        pooling=None)

In [ ]:
class layer_group(object):

  def __init__(self):
          self.conv_layer = []
          self.addition_layer = []
          self.non_conv_layers = []
          self.all_layers = []
          self.total_comp_mac = 0
          self.group_comp_cycles = 0
          self.group_mem = 0
          self.is_DMA = False

  def add_layer(self, layer):
          if layer.layer_type == 'conv':
              self.conv_layer.append(layer)
          elif layer.layer_type == 'add':
              self.addition_layer.append(layer)
          else:
              self.non_conv_layers.append(layer)
          if layer.layer_type == 'dma':
              self.is_DMA = True
          self.all_layers.append(layer)
          self.total_comp_mac += layer.mac

  def is_present(self, layer):
        for t in self.all_layers:
            if layer.parent == t.layer_name:
                return True
        return False

In [ ]:
class custom_layer(object):

    def __init__(self, row=[0]*13):
        self.layer_name = row[0]
        self.layer_type = row[1]
        self.mac = float(row[2])
        self.comp = float(row[3])
        self.activation = float(row[4])
        self.division = float(row[5])
        self.ch_in = float(row[6])
        self.data_in = float(row[7])
        self.xy_dimension = (float(row[8]), float(row[8]))
        self.filter_xy_dim = (float(row[9]), float(row[10]))
        self.ch_out = int(row[11])
        self.parent = []

In [ ]:
preprocess = imagenet_utils.preprocess_input
extractor = keras.Model(inputs=model.inputs,
                            outputs=[layer.output
                                     for layer in model.layers])
batch_size = 1
data = np.zeros((batch_size, input_x, input_y, 3))
data = preprocess(data)
features = extractor(data)
# Neurons in a layer is just the product of the shape for a batchcount of 1
neurons = np.zeros(len(features))
for i, feature in enumerate(features):
    neurons[i] = np.prod(feature.shape[1:3])
grouped_layers = []
temp_group = layer_group()
dma_done = False
for i, layer in enumerate(model.layers):
    temp_layer = custom_layer()
    temp_layer.layer_name = layer.name
    if hasattr(layer, 'kernel'):
        dma_done = True
        grouped_layers.append(temp_group)
        temp_group = layer_group()
        temp_layer.data_in = np.prod(layer.input_shape[1:])
        temp_layer.ch_in = layer.input_shape[3]
        temp_layer.xy_dimension = (layer.input_shape[1], layer.input_shape[2])
        temp_layer.ch_out = layer.output_shape[3]
        temp_layer.layer_type = 'conv'
        temp_layer.mac = np.prod(layer.kernel.shape) * neurons[i]
        temp_layer.filter_xy_dim = (layer.kernel.shape[0], layer.kernel.shape[1])
        if type(layer._inbound_nodes[0].inbound_layers) is list:
            for ib_layer in layer._inbound_nodes[0].inbound_layers:
                temp_layer.parent.append(ib_layer.name)
        else:
            temp_layer.parent = layer._inbound_nodes[0].inbound_layers.name
        temp_group.add_layer(temp_layer)
    else:
        if dma_done is False:
            temp_layer.layer_type = 'dma'
            temp_layer.ch_out = 1
            temp_layer.parent = 'null'
        else:
            temp_layer.ch_out = layer.output_shape[3]
            if type(layer.input_shape) is list:
                for x in layer.input_shape:
                    temp_layer.xy_dimension = (x[1], x[2])
                    temp_layer.ch_in = x[3]
            else:
                temp_layer.data_in = np.prod(layer.input_shape[1:])
                temp_layer.ch_in = layer.input_shape[3]
                temp_layer.xy_dimension = (layer.input_shape[1], layer.input_shape[2])

            if type(layer._inbound_nodes[0].inbound_layers) is list:
                temp_layer.layer_type = 'add'
                temp_layer.data_in = {}
                for ib_layer in layer._inbound_nodes[0].inbound_layers:
                    temp_layer.parent.append(ib_layer.name)
                    temp_layer.data_in[ib_layer.name] = \
                        np.prod(ib_layer.output_shape[1:])
            else:
                temp_layer.parent = \
                    layer._inbound_nodes[0].inbound_layers.name
        lock = 0
        for group in grouped_layers:
            if group.is_present(layer=temp_layer):
                group.add_layer(temp_layer)
                lock = 1
        if lock == 0:
            temp_group.add_layer(temp_layer)

if temp_group != []:
    grouped_layers.append(temp_group)
t = 0
t_1 = 0
for i, group in enumerate(grouped_layers):
        for lyr in group.all_layers:
            t += lyr.mac
        t_1 += group.total_comp_mac
        
print(t)
print(t_1)

1826406400.0
1826406400.0
